In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will use a toy dataset from sklearn.

In [2]:
from sklearn import datasets
from helpers import * 

#Load dataset
sklearn_dataset = datasets.load_breast_cancer()
Xx  = sklearn_dataset.data
y = sklearn_dataset.target * 2 - 1    # labels must be in {-1, 1} for the hinge loss
X = np.ones((Xx.shape[0], Xx.shape[1] + 1 ))    # add a column of ones for intercept
X[:, :-1] = Xx
print("(N, D) =", X.shape)

(N, D) = (569, 31)


## Prepare cost and prediction functions

In [3]:
def calculate_primal_objective(y, X, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer.
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    hinge = np.clip(1-y*(X@w), 0, np.inf)
    return np.sum(hinge)+lambda_/2*np.sum(np.power((w),2))

In [4]:
def calculate_accuracy(y, X, w):
    """compute the training accuracy on the training set (can be called for test set as well).
    X: the full dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    """
    y_pred = predict_labels(w, X)
    return np.mean(y_pred==y)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [5]:
def is_not_zero(x_n, y_n, w):
    return y_n * x_n @ w < 1

def calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples):
    """compute the stochastic gradient of loss plus regularizer.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the index of the (one) datapoint we have sampled
    num_examples: N
    """
    
    # Be careful about the constant N (size) term!
    # The complete objective for SVM is a sum, not an average as in earlier SGD examples!
    
    x_n, y_n = X[n], y[n]
    grad = - y_n * x_n.T if is_not_zero(x_n, y_n, w) else np.zeros_like(x_n.T)
    grad = num_examples * grad + lambda_*w
    
    return grad

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [6]:
def sgd_for_svm_demo(y, X):
    
    max_iter = 2 * int(1e5)
    gamma = 1e-4
    lambda_ = int(1e4)   # big because scales with N due to the formulation of the problem (not an averaged loss)

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        grad = calculate_stochastic_gradient(y, X, w, lambda_, n, num_examples)
        w -= gamma/(it+1) * grad
        
        if it % 10000 == 0:
            cost = calculate_primal_objective(y, X, w, lambda_)
            print("iteration={i}, cost={c}".format(i=it, c=cost))
    
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

sgd_for_svm_demo(y, X)

iteration=0, cost=27903095.95542059
iteration=10000, cost=278.02911883753285
iteration=20000, cost=191.9232499750167
iteration=30000, cost=175.6715978994568
iteration=40000, cost=155.80834093847827
iteration=50000, cost=208.05631754785648
iteration=60000, cost=141.99889133834068
iteration=70000, cost=135.42471050704634
iteration=80000, cost=160.3995148352525
iteration=90000, cost=159.04280400665564
iteration=100000, cost=147.19462160909274
iteration=110000, cost=156.35772883680633
iteration=120000, cost=162.0068331183956
iteration=130000, cost=130.6104863447405
iteration=140000, cost=144.12616905197405
iteration=150000, cost=144.29447386732747
iteration=160000, cost=128.6832175348942
iteration=170000, cost=141.80602628865506
iteration=180000, cost=132.49209540919304
iteration=190000, cost=131.2923814764557
training accuracy = 0.9138840070298769


## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [7]:
def calculate_coordinate_update(y, X, lambda_, alpha, w, n):
    """compute a coordinate update (closed form) for coordinate n.
    X: the dataset matrix, shape = (num_examples, num_features)
    y: the corresponding +1 or -1 labels, shape = (num_examples)
    w: shape = (num_features)
    n: the coordinate to be updated
    """
    x_n, y_n = X[n], y[n]
    old_alpha_n = np.copy(alpha[n])
    gamma = (y_n*w.T@x_n - 1)
    
    if old_alpha_n==0:
        gamma = min(0.0, gamma)
    elif old_alpha_n==1:
        gamma = max(0.0, gamma)
    else:
        gamma = gamma

    alpha[n]=min(
        max(old_alpha_n - gamma*lambda_/(x_n.T@x_n),0)
        ,1)
    w += 1/lambda_*(alpha[n]-old_alpha_n)*x_n*y_n
    return w, alpha

In [8]:
def calculate_dual_objective(y, X, w, alpha, lambda_):
    """calculate the objective for the dual problem."""
    return np.sum(alpha) - lambda_ / 2.0 * np.sum(w ** 2)

In [9]:
def coordinate_descent_for_svm_demo(y, X):
    max_iter = 2*int(1e5)
    lambda_ = int(1e4)   # use same lambda as before in order to compare

    num_examples, num_features = X.shape
    w = np.zeros(num_features)
    alpha = np.zeros(num_examples)
    
    for it in range(max_iter):
        # n = sample one data point uniformly at random data from x
        n = random.randint(0,num_examples-1)
        
        w, alpha = calculate_coordinate_update(y, X, lambda_, alpha, w, n)
            
        if it % 10000 == 0:
            # primal objective
            primal_value = calculate_primal_objective(y, X, w, lambda_)
            # dual objective
            dual_value = calculate_dual_objective(y, X, w, alpha, lambda_)
            # primal dual gap
            duality_gap = primal_value - dual_value
            print('iteration=%i, primal:%.5f, dual:%.5f, gap:%.5f'%(
                    it, primal_value, dual_value, duality_gap))
    print("training accuracy = {l}".format(l=calculate_accuracy(y, X, w)))

coordinate_descent_for_svm_demo(y, X)

iteration=0, primal:1353.30283, dual:0.04942, gap:1353.25341
iteration=10000, primal:153.91661, dual:30.56949, gap:123.34712
iteration=20000, primal:138.10314, dual:55.21300, gap:82.89014
iteration=30000, primal:154.73008, dual:74.86316, gap:79.86692
iteration=40000, primal:131.45037, dual:89.35775, gap:42.09262
iteration=50000, primal:127.48179, dual:98.51894, gap:28.96285
iteration=60000, primal:128.32375, dual:104.69538, gap:23.62837
iteration=70000, primal:131.36238, dual:108.80028, gap:22.56210
iteration=80000, primal:163.28518, dual:111.81499, gap:51.47019
iteration=90000, primal:124.45221, dual:114.36417, gap:10.08804
iteration=100000, primal:134.48530, dual:115.98487, gap:18.50043
iteration=110000, primal:123.42293, dual:117.22538, gap:6.19755
iteration=120000, primal:124.13472, dual:118.13647, gap:5.99824
iteration=130000, primal:125.45809, dual:118.80970, gap:6.64838
iteration=140000, primal:123.13375, dual:119.40269, gap:3.73107
iteration=150000, primal:122.74884, dual:119.8